Modelado de decisiones
===

* *60 min* | Ultima modificación: Junio 22, 2019.

En Analytics, otro de los casos reales de aplicación de la teoría de la probabilidad es el uso de árboles de decisión para representar estados probabilísticos del mundo y analizar sus posibles resultados. Esta técnica, que viene del área de análisis de decisiones en investigación de operaciones, no debe confundirse con los árboles de decisión en aprendizaje estadístico, los cuales son usados para resolver problemas de clasificación.

## Definición del problema

La compañía A va a participar en una convocatoria por invitación directa para el desarrollo y producción de una nueva pieza de equipo para un mercado emergente. Los costos de producción son inciertos. Adicionalmente, se sabe que la compañía B también ha sido invitada a la licitación privada y tiene una experiencia y reputación equivalente a la compañía A. ¿Cómo debe definirse el valor de la oferta por parte de la compañia A?

El gerente de la compañía A debe decidir si la empresa se presenta o no a la licitación privada, y en caso de presentarse puede elegir entre 300, 500 o 700 como valor de su oferta. El equipo comercial estima que la compañía B podría presentar valores de oferta de 400, 600 o 800 con probabilidades de 35%, 50% y 15% respectivamente. El equipo de ingenería estima que los costos podrían ser de 200, 400 o 600 con probabilidades de 25%, 50% y 25% respectivamente. La utilidad para la compañía A se puede estimar simplemente como OFERTA_A - COSTO_A. 

## Solución

Una forma de estructurar el análisis anterior es mediante el uso de árboles de decisión. Cada nodo del árbol representa una variable del problema y las ramas del nodo representa los valores que puede tomar dicha variable. En el problema descrito existen dos variables probabilisticas: la oferta de B (OFERTA_B) y los costos de A (COSTO_A); estas se representan como nodos de la siguiente forma:

           COSTO_A              OFERTA_B
          
         PROB  VAL             PROB  VAL
       __0.25__200           __0.35__400
    __C__0.50__400        __C__0.50__600
      |__0.25__600          |__0.15__800
      
      
Por otra parte, el nodo que representa el valor de oferta de la compañía A es un nodo de decisión, esto es, el gerente puede decidir la acción a tomar (la rama a seguir). Este nodo se representa como:

           OERTA_A
          
               VAL
       ________300
      |________500
    __D
      |________700
      |______NOBID


Para resolver este problema se usará la librería `hardDecisions` de Python. El lenguaje R no tiene una librería equivalente para resolver este problema.

In [3]:
##
## Instale y carge hardDecisions usando pip
##
# !pip3 install hardDecisions
from hardDecisions import *

In [5]:
##
## El primer paso consiste en crear un arbol
##
tree = DecisionTree()

In [14]:
##
## A continuación se crean los nodos. El primer nodo
## creado es el 0, el segundo es el 1 y asi sucesivamente.
##
## El parámetro branches para los nodos de decisión
## recibe una lista de tuplas donde el primer valor es 
## valor del nodo y el segundo es el número del nodo siguiente
##

## Este es el nodo 0
tree.decision_node(name='OFERTA_A',
                   branches=[(300, 1),  ## indica que sigue el nodo 1
                             (500, 1),
                             (700, 1)],
                   max=True)

In [14]:
##
## El parámetro branches para el nodo probabilistico recibe 
## la probabilidad, luego el valor de ese nodo y finalmente
## el índice del nodo siguiente.
##

## Este es el nodo 1
tree.chance_node(name='OFERTA_B',
                 branches=[(35.0,  400,  2), ## indica que sigue el nodo 1
                           (50.0,  600,  2),
                           (15.0,  800,  2)])

## Este es el nodo 2
tree.chance_node(name='COSTO',
                 branches=[(25.0,  200,  3),
                           (50.0,  400,  3),
                           (25.0,  600,  3)])

De acuerdo con el enunciado del problema, si la compañía B presenta una oferta más baja, la compañía A pierde el contrato y su utilidad es cero. Si la compañia A gana el contrato, entonces la utilidad será OFERTA_A - COSTO_A. 

El valor esperado de un nodo probabilistico es la sumatoria de los valores esperados de sus ramas multiplicados por la probabilidad de la correspondiente rama.

In [14]:
##
## Se computa el valor del nodo terminan en función de 
## los demas nodos como una ecuación.
##

## Este es el nodo 3.
tree.terminal_node(expr='(OFERTA_A-COSTO) * (1 if OFERTA_A < OFERTA_B else 0)')

##
## Se imprimen los nodos
##
tree.display_nodes()

Node 0
    Type: DECISION - Maximum Payoff
    Name: OFERTA_A
    Branches:
                         Value  Next Node
                       300.000  1
                       500.000  1
                       700.000  1

Node 1
    Type: CHANCE
    Name: OFERTA_B
    Branches:
          Chance         Value  Next Node
           35.00       400.000  2
           50.00       600.000  2
           15.00       800.000  2

Node 2
    Type: CHANCE
    Name: COSTO
    Branches:
          Chance         Value  Next Node
           25.00       200.000  3
           50.00       400.000  3
           25.00       600.000  3

Node 3
    Type: TERMINAL
    Expr: (OFERTA_A-COSTO) * (1 if OFERTA_A < OFERTA_B else 0)



Ya que las variables son independientes entre si, es posible estructurar su secuencia bajo la forma de un árbol. Ya que las ramas son eventos independientes, es posible calcular la probabilidad de una trayectoria por el árbol, como la multiplicación de las probabilidades de cada rama.  

In [15]:
##
## La función `build_tree` crea el árbol de decisión
## a partir de los nodos creados anteriormente.
##
tree.build_tree()

##
## Imprime el arbol. La [D] indica que es un nodo
## de decisión. La [C] indica que es un nodo 
## probabilistico. La [T] indica que es un nodo terminal
##
tree.display_tree()

|
| #0
\-------[D]
         |
         | #1
         | OFERTA_A=300
         +-------[C]
         |        |
         |        | #2
         |        | OFERTA_B=400
         |        | Prob=35.00
         |        +-------[C]
         |        |        |
         |        |        | #3
         |        |        | COSTO=200
         |        |        | Prob=25.00
         |        |        +-------[T] (OFERTA_A-COSTO) * (1 if OFERTA_A < OFERTA_B else 0)
         |        |        |
         |        |        | #4
         |        |        | COSTO=400
         |        |        | Prob=50.00
         |        |        +-------[T] (OFERTA_A-COSTO) * (1 if OFERTA_A < OFERTA_B else 0)
         |        |        |
         |        |        | #5
         |        |        | COSTO=600
         |        |        | Prob=25.00
         |        |        \-------[T] (OFERTA_A-COSTO) * (1 if OFERTA_A < OFERTA_B else 0)
         |        |
         |        | #6
         |        | OFERTA_B=600
  

In [16]:
## 
## La función `evaluate` hace que las ramas del
## árbol sean evaluadas.
##
tree.evaluate()

##
## Se imprime el arbol evaluado
##
tree.display_tree()

|
| #0
| ExpVal=65.00
| (selected strategy)
\-------[D]
         |
         | #1
         | OFERTA_A=300
         | ExpVal=-100.00
         +-------[C]
         |        |
         |        | #2
         |        | OFERTA_B=400
         |        | Prob=35.00
         |        | ExpVal=-100.00
         |        +-------[C]
         |        |        |
         |        |        | #3
         |        |        | COSTO=200
         |        |        | Prob=25.00
         |        |        | PathProb=0.00
         |        |        | ExpVal=100.00
         |        |        +-------[T] (OFERTA_A-COSTO) * (1 if OFERTA_A < OFERTA_B else 0)
         |        |        |
         |        |        | #4
         |        |        | COSTO=400
         |        |        | Prob=50.00
         |        |        | PathProb=0.00
         |        |        | ExpVal=-100.00
         |        |        +-------[T] (OFERTA_A-COSTO) * (1 if OFERTA_A < OFERTA_B else 0)
         |        |        |
         |

In [17]:
##
## la función `compute_risk_profile` calcula cada 
## valor discreto posible del arbol y su probabilidad
## asociada
##
tree.compute_risk_profile()

##
## Se imprime el perfil de riesgo del árbol
##
tree.display_tree()

|
| #0
| ExpVal=65.00
| Risk Profile:
|      Value  Prob
|    -100.00 16.25
|       0.00 35.00
|     100.00 32.50
|     300.00 16.25
| (selected strategy)
\-------[D]
         |
         | #1
         | OFERTA_A=300
         | ExpVal=-100.00
         +-------[C]
         |        |
         |        | #2
         |        | OFERTA_B=400
         |        | Prob=35.00
         |        | ExpVal=-100.00
         |        +-------[C]
         |        |        |
         |        |        | #3
         |        |        | COSTO=200
         |        |        | Prob=25.00
         |        |        | PathProb=0.00
         |        |        | ExpVal=100.00
         |        |        +-------[T] (OFERTA_A-COSTO) * (1 if OFERTA_A < OFERTA_B else 0)
         |        |        |
         |        |        | #4
         |        |        | COSTO=400
         |        |        | Prob=50.00
         |        |        | PathProb=0.00
         |        |        | ExpVal=-100.00
         |        | 

---
**Ejercicio.--** Complete las probabilidades de cada rama para el árbol presentado a continuación.


                             OFERTA_B                  COSTO_A
                                             
                            PROB  VAL                PROB  VAL         PROB
                                             
                                                   __0.25__200______ 0.0875
                          __0.35__400_____________C__0.50__400______      ?
                         |                        |__0.25__600______      ?
                         |
                         |                        ___0.25__200_____       ?
                     ____C__0.50__600____________C___0.50__400_____  0.2500
                         |                       |___0.25__600_______     ?
                         |
                         |                        ___0.25__200_______0.0375
                         |__0.15__800____________C___0.50__400_______     ?
                                                 |___0.25__600_______     ?
                        

---

---
**Ejercicio.--** El siguiete diagrama representa el árbol de decisiones del problema enunciado. Realice lo siguiente:

* Compute el valor de la utilidad para todas las ramas terminales.


* Compute el valor esperado de los nodos COSTO_A.


* Compute el valor esperado de los nodos OFERTA_B.


* Compute el valor esperado de la decisión.


* ¿Cuál debe ser la oferta de A?


              DECISION    OFERTA_A           OFERTA_B                  COSTO_A      UTILIDAD
             -------------------------------------------------------------------------------------             
                             
              EXP VAL     VAL                PROB  VAL   EXP VAL      PROB  VAL      EXP VAL    PROB 
                                                                    __0.25__200_______+100.0__0.0875 
                                           __0.35__400____-100.0___C__0.50__400_______     ?__     ?
                                          |                        |__0.25__600_______     ?__     ?
                                          |
                                          |                        ___0.25__200_______     ?__     ?
                        __300_____-100.0__C__0.55__600____     ?__C___0.50__400_______-100.0__0.2500
                       |                  |                       |___0.25__600_______     ?__     ?
                       |                  |
                       |                  |                        ___0.25__200_______     ?__     ?
                       |                  |__0.15__800____     ?__C___0.50__400_______     ?__     ? 
                       |                                          |___0.25__600_______-300.0__0.0375
                       |
                       |                                            __0.25__200_______   0.0__0.0875
                       |                   __0.35__400____-100.0___C__0.50__400_______     ?__     ?
                       |                  |                        |__0.25__600_______     ?__     ?
                       |                  |
                       |                  |                        ___0.25__200_______     ?__     ? 
                       |__500______    ?__C__0.55__600____-100.0__C___0.50__400_______+100.0__0.2500
                       |                  |                       |___0.25__600_______     ?__     ?
                       |                  |
                       |                  |                        ___0.25__200_______     ?__     ?
                       |                  |__0.15__800____-100.0__C___0.50__400_______     ?__     ?
                       |                                          |___0.25__600_______-100.0__0.0375
           _______  ?__D
                       |                                            __0.25__200_______   0.0__0.0875
                       |                   __0.35__400____-100.0___C__0.50__400_______     ?__     ? 
                       |                  |                        |__0.25__600_______     ?__     ?
                       |                  |
                       |                  |                        ___0.25__200_______     ?__     ?
                       |__700_______   ?__C__0.55__600____-100.0__C___0.50__400_______   0.0__0.2500
                       |                  |                       |___0.25__600_______     ?__     ?
                       |                  |
                       |                  |                        ___0.25__200_______     ?__     ?
                       |                  |__0.15__800____-100.0__C___0.50__400_______     ?__     ?
                       |                                          |___0.25__600_______+100.0__0.0375
                       |
                       |__NOBID______0.0____________________________________________________

---

**Ejercicio.--** A partir de los valores esperados de los nodos terminales del árbol y su respectiva probabilidad, grafique la curva de probabilidad acumulada para la utilidad de la compañía A.
